In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import RGBNirBioCountryModule, RGBNirBioCountryDataModule

dm = RGBNirBioCountryDataModule(batch_size=512, num_workers=10, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [3]:
model = RGBNirBioCountryModule.load_from_checkpoint('checkpoints/r18RGBNirBioCountry-val_loss=6.34389-epoch=1.ckpt')
model.hparams

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9d52505af0>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1289, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9d52505af0>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1289, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdo

"backbone":         resnet18
"bio_dropout":      0.0
"bio_layers":       [256, 512]
"datamodule":       {'batch_size': 512, 'num_workers': 10, 'pin_memory': True}
"early_stopping":   True
"num_bio":          28
"optimizer":        Adam
"optimizer_params": {'lr': 0.001}
"pretrained":       True
"trainer":          {'gpus': 1, 'max_epochs': 30, 'logger': <pytorch_lightning.loggers.wandb.WandbLogger object at 0x7f9d364721f0>, 'enable_checkpointing': True, 'overfit_batches': 0, 'deterministic': True, 'precision': 16, 'callbacks': [<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f9d36472220>, <pytorch_lightning.callbacks.early_stopping.EarlyStopping object at 0x7f9d050a1550>, <pytorch_lightning.callbacks.progress.tqdm_progress.TQDMProgressBar object at 0x7f9d050a1580>, <pytorch_lightning.callbacks.model_summary.ModelSummary object at 0x7f9d050a15b0>, <pytorch_lightning.callbacks.gradient_accumulation_scheduler.GradientAccumulationScheduler object at 0x7f9d050a15e0

In [4]:
from tqdm import tqdm
from src.GLC.metrics import top_30_error_rate
import numpy as np 

model.cuda(1)
dl = dm.val_dataloader()
accs = []
for batch in tqdm(dl):
    labels = batch['label']
    preds = model.predict(batch)
    accs.append(top_30_error_rate(labels, preds.cpu()))
np.mean(accs)

100%|██████████| 79/79 [00:29<00:00,  2.63it/s]


0.7105199542897327

In [5]:
from tqdm import tqdm

model.cuda(1)
dl = dm.test_dataloader()
labels, observations = [], []
for batch in tqdm(dl):
    preds = model.predict(batch)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observation_ids = batch['observation_id']
    observations += observation_ids.numpy().tolist()

100%|██████████| 72/72 [00:23<00:00,  3.10it/s]


In [6]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
29370,21312108,2159 720 5619 2196 5045 5698 570 5366 2992 389...
29319,21307602,5274 2143 6286 2362 10096 5005 6902 9087 2668 ...
5007,10795864,1041 692 687 103 49 397 188 660 192 380 24 29 ...
28401,21208724,125 396 2196 5619 5974 2200 720 1753 2268 1863...
25100,20854260,2575 836 5189 5207 5523 5940 5565 5025 3705 30...


In [7]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
9156,10435802,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
31549,21546623,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
20822,20385967,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
10982,10527192,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
31744,21567083,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
24639,20802988,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
7342,10347789,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
20496,20352694,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
32905,21688056,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
35360,21956146,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
